In [1]:
import boto3
import pandas as pd
import os
import io
import sys

# Configuration Access Keys
current_dir = os.getcwd()
project_root_dir = os.path.dirname(current_dir)
sys.path.insert(0, project_root_dir)

from steps.s3_ingest_data import *
from steps.clean_data import *
from config.access_keys import *
import pandas as pd
import logging
if __name__ == "__main__":
    logging.info("Starting S3CSVReader...")
    reader = S3CSVReader(bucket_name=S3_BUCKET_NAME, region_name=AWS_REGION, 
                         aws_access_key_id=S3_AWS_ACCESS_KEY_ID, aws_secret_access_key=S3_AWS_SECRET_ACCESS_KEY)
    # Read the CSV file from S3
    df = reader.read_csv(s3_key=S3_KEY, encoding='utf-8')
    
    # Clean, transform, and split the data.
    processed_df = clean_data(df)
    X_train, X_val, y_train, y_val = split_data(processed_df)

2025-08-12 18:15:05,986 - root - INFO - Starting S3CSVReader...
2025-08-12 18:15:05,987 - steps.s3_ingest_data - INFO - Initializing S3 client with explicit AWS access keys.
2025-08-12 18:15:06,343 - steps.s3_ingest_data - INFO - S3CSVReader initialized for bucket 'housepred-data' in region 'us-east-1'.
2025-08-12 18:15:06,344 - steps.s3_ingest_data - INFO - Attempting to read 'House_Price_Prediction_Dataset.csv' from bucket 'housepred-data' with encoding 'utf-8'.
2025-08-12 18:15:06,563 - steps.s3_ingest_data - INFO - Successfully fetched object 'House_Price_Prediction_Dataset.csv'.
2025-08-12 18:15:06,599 - steps.s3_ingest_data - INFO - Successfully loaded 'House_Price_Prediction_Dataset.csv' into DataFrame. Shape: (2000, 10)
2025-08-12 18:15:06,600 - root - INFO - Starting data cleaning process...
2025-08-12 18:15:06,607 - root - INFO - Data preprocessing completed successfully.
2025-08-12 18:15:06,608 - root - INFO - Dividing data into training and validation sets...
2025-08-12 18:

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.ensemble import RandomForestRegressor
import mlflow

# Set tracking URI and initialize MLflow autologging
mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.autolog(silent=True)

# Define the search space for hyperparameters
space = {
    'n_estimators': hp.choice('n_estimators', range(10, 300)),
    'max_depth': hp.choice('max_depth', range(1, 20)),
    'min_samples_split': hp.uniform('min_samples_split', 0.1, 1.0),
    'min_samples_leaf': hp.choice('min_samples_leaf', range(1, 10))
}

# Define objective function
def objective(params):
    with mlflow.start_run(nested=True):
        reg = RandomForestRegressor(
            n_estimators=params['n_estimators'],
            max_depth=params['max_depth'],
            min_samples_split=params['min_samples_split'],
            min_samples_leaf=params['min_samples_leaf'],
            random_state=42
        )
        
        reg.fit(X_train, y_train)
        r2_score = reg.score(X_val, y_val)

        return {'loss': -r2_score, 'status': STATUS_OK}

# Run optimization
trials = Trials()
with mlflow.start_run(run_name='hyperopt_optimization_regressor'):
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=50,
        trials=trials
    )

print("Best parameters found:", best)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

2025-08-12 18:24:56,715 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.004969 seconds
2025-08-12 18:24:56,715 - hyperopt.tpe - INFO - TPE using 0 trials
2025/08/12 18:24:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/tawate/Documents/HousePredictionMLPipeline/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-intege

🏃 View run youthful-gnat-961 at: http://127.0.0.1:8080/#/experiments/0/runs/ff228bf9e0d446b89bb90d00c32f6dad

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0

  2%|▏         | 1/50 [00:07<06:14,  7.64s/trial, best loss: 0.0006643596864430723]

2025-08-12 18:25:04,359 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.007073 seconds
2025-08-12 18:25:04,360 - hyperopt.tpe - INFO - TPE using 1/1 trials with best loss 0.000664
2025/08/12 18:25:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/tawate/Documents/HousePredictionMLPipeline/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/m

🏃 View run redolent-squid-972 at: http://127.0.0.1:8080/#/experiments/0/runs/47b70288e2674c1fa638eb95219bd1df

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0                       

  4%|▍         | 2/50 [00:12<04:43,  5.90s/trial, best loss: 0.0006643596864430723]

...

🏃 View run skittish-steed-391 at: http://127.0.0.1:8080/#/experiments/0/runs/4c4ca47877f44b2db169678c8b55efe2

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0                        

 96%|█████████▌| 48/50 [03:41<00:08,  4.35s/trial, best loss: 0.0006643596864430723]

2025-08-12 18:28:37,767 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.005875 seconds
2025-08-12 18:28:37,768 - hyperopt.tpe - INFO - TPE using 48/48 trials with best loss 0.000664
2025/08/12 18:28:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/tawate/Documents/HousePredictionMLPipeline/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest

🏃 View run judicious-rook-60 at: http://127.0.0.1:8080/#/experiments/0/runs/0edcf3f8843645caab56c4f0402a912b

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0                        

 98%|█████████▊| 49/50 [03:45<00:04,  4.37s/trial, best loss: 0.0006643596864430723]

2025-08-12 18:28:42,181 - hyperopt.tpe - INFO - build_posterior_wrapper took 0.007526 seconds
2025-08-12 18:28:42,182 - hyperopt.tpe - INFO - TPE using 49/49 trials with best loss 0.000664
2025/08/12 18:28:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/tawate/Documents/HousePredictionMLPipeline/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest

🏃 View run tasteful-snail-194 at: http://127.0.0.1:8080/#/experiments/0/runs/7be7234592e54cc484973553445030fa

🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0                        

100%|██████████| 50/50 [03:49<00:00,  4.59s/trial, best loss: 0.0006643596864430723]
🏃 View run hyperopt_optimization_regressor at: http://127.0.0.1:8080/#/experiments/0/runs/fc0bc262b6b84ffd9dc57498fb126629
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0
Best parameters found: {'max_depth': np.int64(4), 'min_samples_leaf': np.int64(8), 'min_samples_split': np.float64(0.7429712813822219), 'n_estimators': np.int64(58)}


In [5]:
best

{'max_depth': np.int64(4),
 'min_samples_leaf': np.int64(8),
 'min_samples_split': np.float64(0.7429712813822219),
 'n_estimators': np.int64(58)}